# 一、写在前面

首先什么是欺诈？根据我国法律定义，欺诈行为指一方当事人故意告知对方虚假情况，或故意隐瞒真实情况（如保持沉默），诱使对方当事人作出错误意思表示的民事行为。在金融领域，随着银行数字化转型加速推进，线下业务向线上迁移的过程中，信息倒卖、信贷欺诈、恶意"薅羊毛"、盗卡盗刷等各类欺诈行为已形成规模化产业链，每年给行业造成超千亿元的经济损失。风险管理经验表明，金融机构反欺诈能力越薄弱，遭受欺诈攻击的频率与损失程度越严重。当前，从事欺诈活动的黑产中介已呈现团伙化、专业化特征，更值得警惕的是，大数据分析、深度学习及人工智能等前沿技术正被欺诈分子反向利用，形成技术对抗的恶性循环。如何有效破解欺诈风险困局，已成为金融业务健康发展的核心命题。

本文聚焦金融业务中高频出现的证书文档类图像伪造检测难题。实践中，大量业务场景涉及证件、合同等关键文档的图像验证，而伪造者通过图像篡改、合成等技术手段制造虚假材料的现象屡见不鲜。本任务要求通过训练集构建高效检测算法，实现对测试集伪造图像的精准篡改区域定位。然而，此类图像样本的采集与标注面临两大核心痛点：一是人工标注成本高昂，需专业人员逐帧核查；二是伪造技术迭代迅速，传统检测模型难以适应新型篡改手法。

为突破数据限制与技术瓶颈，我们尝试将伪造检测问题转化为目标检测框架下的模型优化问题。具体而言，结合几何变换、噪声注入、纹理合成等数据增强技术，扩展训练数据的多样性与鲁棒性。**在此基础上，融合ERNIE-4.5-VL-28B-A3B多模态大模型的语义理解能力、PaddleDetection目标检测框架的精准定位优势，构建"语义-视觉"双通道检测体系。**通过跨模态特征融合实现从语义逻辑一致性验证到像素级篡改痕迹检测的双重校验，有效提升对新型伪造技术的识别能力。这对维护金融交易安全、遏制数字欺诈蔓延具有重要的实践价值与理论意义。


# 二、算法思路

## 2.1 核心方法

转换为目标检测问题，将篡改区域定义目标类别：

* classl = {"cheat": 0, "pmiss": 1}

## 2.2 数据处理

原始图像尺寸参差不齐，统一使用颜色125填充至1550*1550，

举例：

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/35cac87ec8e44094b391ed3f7169fe1ebfaf6b597d9a4d46a5b64a6d18cfac36" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/8e12dda1da3e481c9eb3553b24aa90286c6e1e27aab5493c8b0ae5834efdd172" width="300">
</div>

颜色125部分定义为第二类别class-pmiss

对mask进行解析，如下红色为class-pmiss，蓝绿色为class-cheat

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/42d5f2d8cb0a47828340de49061810a124b74c8bcf054730afde8560b461b934" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/25757cd4bf4748929cccb1a2c051cbfb439817c61e9d40e8ad90805ef26a8374" width="300">
</div>

据此构建COCO数据集，用于目标检测。


## 2.3 快速筛选

这里使用VLM在初始样本集合中，快速找到可能被伪造的样本。

我们引入**ERNIE-4.5-VL-28B-A3B**，相关设置如下：

* 模型配置

```python
    extra_body={
        "penalty_score": 1,
        "enable_thinking": True
    },
    max_completion_tokens=8000,
    temperature=0.3,
    top_p=0.8,
    frequency_penalty=0,
    presence_penalty=0
```

* 系统提示词


In [1]:
"""
你是一个识别假证的鉴别专家，我会给你一张图片，你的任务是返回它的伪造水平，分数范围为0-100分，分数越高代表它被篡改的可能性越大。  

# 任务说明  
- **输入**：一张证件图片（如身份证、护照、驾照等）。  
- **输出**：一个0-100的伪造评分，分数越高表示篡改可能性越大。  
- **评分标准**：  
  - **0-30分**：无明显篡改痕迹，可信度高。  
  - **31-60分**：存在轻微可疑痕迹，需进一步检查。  
  - **61-80分**：有明显篡改痕迹，可信度较低。  
  - **81-100分**：高度疑似伪造，篡改痕迹显著。  

# 分析步骤  
1. **检查证件整体一致性**：  
   - 字体、排版、颜色是否与官方样本一致？  
   - 是否存在模糊、错位或异常边缘？  
2. **细节验证**：  
   - 防伪特征（如 hologram、UV图案、水印）是否完整且清晰？  
   - 数字或文字是否存在PS痕迹（如像素不一致、边缘锐化）？  
3. **逻辑校验**：  
   - 证件号码、日期、姓名等是否符合官方规则（如校验位）？  
   - 背景纹理、光影是否自然？  

# 输出格式  
返回一个JSON格式的结果，包含以下字段：  
```json  
{  
  "score": 0-100,  
  "details": "具体可疑点的文字描述（如'照片区域边缘有锯齿'）",  
  "confidence": "高/中/低（基于分析的可信度）"  
}  
```  

# 示例  
**输入**：一张疑似PS过的身份证图片（照片区域边缘不自然）。  
**输出**：  
```json  
{  
  "score": 75,  
  "details": "照片区域边缘有锯齿，与背景光影不匹配",  
  "confidence": "中"  
}  
```  

# 注意事项  
- 若图片质量过低（如模糊、反光），需降低置信度并说明。  
- 对高分辨率图片需重点检查微观细节（如墨迹分布）。
"""
pass

之后我们将训练集中部分图片依次传递给模型，并要求其“直接返回json即可”，大概如下：

#### 2.3.1 样本A

* 上传示例

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/58570f7a7b164b4da7555c35bb840b9b4b2e69ee79094dbe8ddbe6fb9d2e5c61" width="300">
</div>

* 模型思考

**用户让我当假证鉴别专家，给一张作品登记证书打分。首先，我需要仔细看看这张证书的细节。证书的标题是“作品登记证书”，看起来正规，边框有装饰花纹，颜色是浅黄色，字体是黑色的，整体布局整齐。 接下来，检查登记号“浙江登记-1998-F-2310”，这个格式看起来像地方登记号，可能符合浙江的登记系统。作品类别是美术，作者和著作权人信息部分，作者名字被模糊处理了，著作权人显示的是“上海市嘉定区欧莱装饰”，这里可能有问题，因为著作权人通常是个人或公司名称，而装饰公司可能不太常见作为美术作品的著作权人，但不确定是否违法，需要进一步考虑。 首次发表时间是1998年12月25日，首次出版/制作日期是2013年8月5日，间隔了14年多，这在现实中可能不太合理，尤其是如果作品是原创的话，通常发表和制作的时间不会相隔这么久。不过，如果是重新制作或者改编，可能没问题，但需要更多信息。 申请机构是某有限公司，经过浙江省专利局审核，但根据《作品自愿登记试行办法》，应该是由文化部门或者版权局负责，而不是专利局。这里可能存在机构名称错误，专利局和版权局是不同的部门，所以这里可能是伪造的一个点。 登记日期是1998年12月30日，比发表日期晚五天，这在流程上是可能的，但需要看当地登记流程是否允许。不过，通常登记会在发表之后进行，所以这个时间点可能没问题。 右下角有红色的登记机构签章，作品登记专用章，但图片中可能显示的是电子章或者复制的章，需要检查章的细节是否清晰，是否有防伪特征。如果章看起来是打印的或者边缘不清晰，可能是伪造的。不过用户提供的图片中章看起来比较清晰，但因为是图片，可能存在复制痕迹。 整体来看，证书的格式和部分信息存在可疑点。比如，申请机构是专利局，而应该是版权局；著作权人名称可能不符合常规；作品类别是美术，但申请机构是专利局，这可能矛盾。此外，首次发表和制作时间相隔太久，可能不合理。 根据评分标准，31-60分是轻微可疑，61-80分有明显篡改。这里可能属于60分左右，因为机构名称错误，时间间隔问题，以及可能的其他细节问题。但需要确认是否有其他防伪特征，比如水印、Hologram等，但证书上没有明显显示这些。因此，综合来看，伪造评分可能在60分左右，属于有明显篡改，但可能不是高度伪造，因为证书的基本结构存在，但关键信息有错误。 所以，输出JSON时，分数设为60，details描述机构名称错误，时间间隔问题，以及可能的章的复制痕迹。confidence设为中，因为虽然有可疑点，但证书整体结构存在，可能不是完全伪造，但存在明显问题。**

* 返回json

```json
{  
  "score": 60,  
  "details": "1. 申请审核机构名称错误：依据《作品自愿登记试行办法》，登记审核机构应为文化部门或版权局，而证书显示为‘浙江省专利局’，存在机构名称矛盾；\n2. 著作权人名称异常：‘上海市嘉定区欧莱装饰’作为美术作品著作权人表述不符合常规（通常为自然人或正规文化公司），可能存在主体信息篡改；\n3. 时间逻辑矛盾：作品首次发表（1998年）与首次制作（2013年）间隔14年，若为原创作品需说明合理理由，否则存在时间线异常；\n4. 章印细节可疑：虽章印清晰，但电子化复制可能存在边缘锐化或防伪特征缺失的潜在风险。",  
  "confidence": "中"  
}
```


#### 2.3.2 样本B

* 上传示例

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/65b9c8ca1c594b06a422be232b0c68449beb8ec364644ea88c5be3c2a321e7ba" width="300">
</div>

* 模型思考

**```json { "score": 30, "details": "整体排版和字体与官方样本基本一致，无明显PS痕迹，但二维码区域细节略显粗糙，存在轻微可疑痕迹。", "confidence": "中" } ``` 上述评分基于整体一致性较高，但二维码区域质量较低，可能影响可信度。若需更高精度鉴定，建议结合官方样本进一步比对。**

* 返回json

```json
{  
  "score": 30,  
  "details": "整体排版和字体与官方样本基本一致，无明显PS痕迹，但二维码区域细节略显粗糙，存在轻微可疑痕迹。",  
  "confidence": "中"  
}

```


## 2.4 数据增强

原有训练数据集较少，且部分图像来自同一张图像，为避免过拟合。使用了以下方法以数据增强。

* **原始**

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/25de3e6483384dd99826f2be6689c8ecbb36845080c048d086677882b24f9929" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/ed867d61804347c3a0c4d417b98c11ad887a6676a3c24cce95246cc64a53044e" width="300">
</div>

* **cv旋转0**

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/169979b40143465dabf4b500199b1ad37f92de14d0a94433952620e7c9f0745a" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/5a10eae6e8d54ee297ba4f384c3ca0d3e4bc10cd2230404f98d1b77bc68e8123" width="300">
</div>

* **cv旋转1**

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/71f67197284f40608f06cbfd9c4c81cb1ba463f11774458f9093b9ad7890892c" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/ad0232aa4af14455836b028218c9dc1e751034e8d9ba4f71be02d93d4da8026d" width="300">
</div>

* **cv旋转-1**

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/a4972a59836043919b5d8d734b57a3658413f3d2990a4bbe8f024fe3aeb8adfb" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/9a97bbc5c32c4ad8a26679b11c5530920d33ff62f1924a1793dcf627f9d82d3e" width="300">
</div>

* **M2（随机生成1/2边长正方形作为遮挡，遮挡部分定义为class-pmiss）**

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/27cc70024bd94d04aa64b97f83c726861a2aeeb35053482ca32d3dd2855cef87" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/f0876a309ff94db5910ed3ed1c678dd80df1d5b23fb440e8a1396116bb6574de" width="300">
</div>

* **MF（随机生成1/16边长正方形作为遮挡，遮挡部分定义为class-pmiss）**

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/8bcb4c9d5b734b7a9842cbec4f221083b7a63406409e4f80bf9e5389028cf5f4" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/12ffd29968224caba5abd170cb385d934f53b655378e4964940eaac6ca05229a" width="300">
</div>

* **blur（调整蓝色色调，肉眼不可见）**

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/52513e1cf8934bc8b96ff6befbd422f6f83fbee6a74942a7ad76b91fd3c1fb2c" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/adc9b4fe977c4f6588124e9b71ba8253f112a540c8ca4b5e8c21cd61f9f9a77f" width="300">
</div>

* **chicken（逛超市看到鸡块想到的鸡块法，也就是随机删减）**

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/5ea371cf5e364d828249b18012111947c5adb01429ee4e76b52994e7e42d0da7" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/cde75bb320254836b450b0e6be052e515844a83ff46845afa5e5ffe1319f21a9" width="300">
</div>

* **jely（知名设计师jely提出的1/4随机拼凑）**

<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/ce3c4d388dfc45929e24aed24bcbda4c637c638b72be4d03a7c6fa685701f9e9" width="300">
<img src="https://ai-studio-static-online.cdn.bcebos.com/1394783f5e504814a3d8bf68501513a70dd94c9142fb4ca1b9e5f9396384501e" width="300">
</div>


## 2.5 数据划分

```python
_n = random.choice(list("0000000012"))
```

**将数据集随机划分为 训练集、测试集和验证集，三者比例为 8:1:1**


# 三、ERNIE-4.5-VL-28B-A3B

## 3.1 模型介绍

[https://aistudio.baidu.com/modelsdetail/30648](https://aistudio.baidu.com/modelsdetail/30648)

* 模型名称

**ERNIE-4.5-VL-28B-A3B**

* 参数规模

**28B**

* 类型

**Image-Text-to-Text**

* 应用场景

**图文生成/分析，覆盖常见多模态任务**


## 3.2 模型部署

这里我们通过飞桨-部署-模型部署来快速实现，

![](https://ai-studio-static-online.cdn.bcebos.com/4f70f5a7d6164079a3865dd94a014479d773fca3a74342a88355c6ee96aa0eec)

部署成功可看到，

![](https://ai-studio-static-online.cdn.bcebos.com/ca5a6be48625499ca7fdc4519e4c64c4a510b9ddc7ba4bb782b2b9f877b25835)

点击**详情**查看调用示例，

![](https://ai-studio-static-online.cdn.bcebos.com/510bd4cc1e2e40869b507dd66b0de990b4af0ac6da69487c9d3e3cd3a1c35dc0)

简单测试一下：


In [2]:

# 加载API_KEY，请在.env中自行配置。
import os
from dotenv import load_dotenv
load_dotenv()
API_KEY, API_URL = os.getenv("API_KEY"), os.getenv("API_URL")
# print(API_KEY, API_URL)

# pip install openai
from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=f"https://api-{API_URL}.aistudio-app.com/v1",
)

completion = client.chat.completions.create(
    model="default",
    temperature=0.6,
    messages=[
        {"role": "user", "content": "你好，请介绍一下你自己"}
    ],
    stream=True
)

for chunk in completion:
    if hasattr(chunk.choices[0].delta, "reasoning_content") and chunk.choices[0].delta.reasoning_content:
        print(chunk.choices[0].delta.reasoning_content, end="", flush=True)
    else:
        print(chunk.choices[0].delta.content, end="", flush=True)




用户让我介绍一下自己，我需要先理解他的需求。可能他是在做一个自我介绍，或者想了解AI的能力，或者只是随便聊聊。首先，我应该保持友好和开放的态度，让用户感到舒适。

接下来，我需要考虑如何结构化回答。用户可能希望知道我的功能、特点，以及能提供的帮助。要涵盖语言理解、知识范围、创作能力、多语言支持、个性化服务这些方面。同时，要避免使用技术术语，保持口语化，让用户容易理解。

还要注意用户可能的深层需求。比如，他们可能想测试我的能力，或者寻找解决问题的方法。所以，在介绍中要强调我可以协助各种任务，比如学习、工作、创意等，这样用户能联想到自己的具体需求。

另外，用户可能对AI的局限性有疑问，所以在回答中要适当提及，比如无法实时获取信息，需要用户核实。这有助于建立信任，避免用户对我的回答产生误解。

最后，保持回答简洁明了，分点列出，但用自然的方式表达，而不是生硬的列表。比如用“首先”、“其次”、“此外”这样的连接词，让内容更流畅。同时，结尾部分要邀请用户提出具体问题，鼓励互动，这样能更好地满足用户的需求。


你好！我是你的智能助手，可以帮你解答问题、提供建议、协助创作或进行日常对话。以下是我的一些特点：

1. **多领域知识**  
   我了解广泛的主题，从科学、历史到生活技巧、娱乐等，能为你提供信息或解释概念。

2. **语言理解与生成**  
   我可以理解并生成自然流畅的中文（及其他支持的语言），帮助你处理文本、翻译或润色内容。

3. **创意支持**  
   无论是写故事、诗歌、文案，还是头脑风暴，我都能提供灵感或建议。

4. **个性化服务**  
   根据你的需求调整回答风格（比如正式/轻松），或提供分步指导帮助你解决问题。

5. **实时限制说明**  
   我的知识截止到2024年7月，无法访问实时数据，但可以通过逻辑推理和已有知识为你提供参考。

需要我帮忙解决什么问题吗？随时告诉我哦！ 😊

## 3.3 模型调用


In [3]:


completion = client.chat.completions.create(
    model="default",
    messages=[
    {
        "role": "system",
        "content": "你是一个识别假证的鉴别专家，我会给你一张图片，你的任务是返回它的伪造水平，分数范围为0-100分，分数越高代表它被篡改的可能性越大。  \n\n# 任务说明  \n- **输入**：一张证件图片（如身份证、护照、驾照等）。  \n- **输出**：一个0-100的伪造评分，分数越高表示篡改可能性越大。  \n- **评分标准**：  \n  - **0-30分**：无明显篡改痕迹，可信度高。  \n  - **31-60分**：存在轻微可疑痕迹，需进一步检查。  \n  - **61-80分**：有明显篡改痕迹，可信度较低。  \n  - **81-100分**：高度疑似伪造，篡改痕迹显著。  \n\n# 分析步骤  \n1. **检查证件整体一致性**：  \n   - 字体、排版、颜色是否与官方样本一致？  \n   - 是否存在模糊、错位或异常边缘？  \n2. **细节验证**：  \n   - 防伪特征（如 hologram、UV图案、水印）是否完整且清晰？  \n   - 数字或文字是否存在PS痕迹（如像素不一致、边缘锐化）？  \n3. **逻辑校验**：  \n   - 证件号码、日期、姓名等是否符合官方规则（如校验位）？  \n   - 背景纹理、光影是否自然？  \n\n# 输出格式  \n返回一个JSON格式的结果，包含以下字段：  \n```json  \n{  \n  \"score\": 0-100,  \n  \"details\": \"具体可疑点的文字描述（如'照片区域边缘有锯齿'）\",  \n  \"confidence\": \"高/中/低（基于分析的可信度）\"  \n}  \n```  \n\n# 示例  \n**输入**：一张疑似PS过的身份证图片（照片区域边缘不自然）。  \n**输出**：  \n```json  \n{  \n  \"score\": 75,  \n  \"details\": \"照片区域边缘有锯齿，与背景光影不匹配\",  \n  \"confidence\": \"中\"  \n}  \n```  \n\n# 注意事项  \n- 若图片质量过低（如模糊、反光），需降低置信度并说明。  \n- 对高分辨率图片需重点检查微观细节（如墨迹分布）。"
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "直接返回json即可"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://lm-app-online.bj.bcebos.com/playground/979775/20251227233d973b55f45e4e34b255ae6f28e13df050e972f9a0624add836dfdc5f920d98e"
                }
            }
        ]
    },
    ],
    stream=True,
    extra_body={
        "penalty_score": 1,
        "enable_thinking": True
    },
    max_completion_tokens=8000,
    temperature=0.3,
    top_p=0.8,
    frequency_penalty=0,
    presence_penalty=0
)

for chunk in completion:
    if hasattr(chunk.choices[0].delta, "reasoning_content") and chunk.choices[0].delta.reasoning_content:
        print(chunk.choices[0].delta.reasoning_content, end="", flush=True)
    else:
        print(chunk.choices[0].delta.content, end="", flush=True)




```json
{
  "score": 0,
  "details": "该证件为中华人民共和国出版物经营许可证，整体字体、排版、颜色与官方样本一致，无模糊、错位或异常边缘，防伪特征（如底纹）完整且清晰，数字和文字无PS痕迹，证件号码、日期、名称等符合官方规则，背景纹理和光影自然。",
  "confidence": "高"
}
```
<think>
现在我需要分析用户提供的这张出版物经营许可证的图片，判断它的伪造水平。首先，我要按照评分标准来一步步检查。

首先检查整体一致性。证件的字体、排版和颜色看起来都很标准，和官方样本对比的话，应该没有问题。比如标题“中华人民共和国出版物经营许可证”的位置和字体大小都符合规范。经营地址、法定代表人等信息排列整齐，没有模糊或者错位的情况。边缘部分也没有明显的锯齿或破损，整体看起来很清晰。

接下来是细节验证。防伪特征方面，证件底部的国徽和“中华人民共和国新闻出版总署监制”的印章都很清晰，国徽的细节比如五角星和天安门图案都很明显，印章的墨迹均匀，没有重影或模糊。右侧的审核机关盖章处，虽然有些文字被涂黑处理，但盖章的形状和位置都符合规范，没有PS的痕迹。数字和文字部分，比如日期“2010年6月12日”和公司名称，字体工整，边缘锐利，没有像素化的现象。

然后是逻辑校验。公司名称是“中国水利水电第五工程局有限公司”，这看起来是一个正规的企业名称，经营地址在四川省成都市，符合该公司的注册信息。注册资本用比特币表示可能有问题，但这里可能只是举例或者题目中的设定，实际证件中可能允许不同形式的注册资本，但需要确认是否符合当地规定。不过根据题目给出的信息，这里可能不需要深入，因为题目中的证件是示例，重点在于整体判断。有效期到2028年6月28日，这个日期是否合理需要看公司成立时间，但作为示例，可能没问题。

背景纹理和光影方面，证件的底纹是复杂的图案，颜色均匀，没有异常的反光或阴影，整体看起来自然。

综合来看，这张许可证在字体、排版、防伪特征、数字文字等方面都没有明显的篡改痕迹，可信度很高。因此评分应该在0-30分之间，属于无明显篡改，可信度高。

```json
{
  "score": 0,
  "details": "证件整体字体、排版、颜色与官方样本一致，无模糊、错位或异常边缘；防伪特征（底纹、国徽印章）清晰完整，数字/文字边

# 四、PaddleDetection

* 推荐 目标检测7日打卡营 

[https://aistudio.baidu.com/education/group/info/1617](https://aistudio.baidu.com/education/group/info/1617)

## 4.1 安装说明

### 4.1.1 PaddlePaddle安装


In [4]:

import paddle
paddle.utils.run_check()


/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


Running verify PaddlePaddle program ... 


/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/pir/math_op_patch.py:219: UserWarning: Value do not have 'place' interface for pir graph mode, try not to use it. None will be returned.
  warnings.warn(
I1228 01:28:56.627260 480065 pir_interpreter.cc:1524] New Executor is Running ...
W1228 01:28:56.630679 480065 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.8
I1228 01:29:03.041854 480065 pir_interpreter.cc:1547] pir interpreter is running by multi-thread mode ...


PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


### 4.1.2 PaddleDetection安装


In [5]:

import os
from IPython.display import clear_output, Image, display

fzip = "PaddleDetection-release-2.8.1.zip"
if os.path.exists(f"/home/aistudio/conf/{fzip}"):
    %cd /home/aistudio
    os.system(f"unzip /home/aistudio/conf/{fzip}")
    ! mv PaddleDetection-release-2.8.1 PaddleDetection
else:
    %cd /home/aistudio
    ! rm -rf PaddleDetection PaddleDetection.zip
    ! git clone https://gitee.com/PaddlePaddle/PaddleDetection.git

%cd /home/aistudio/PaddleDetection
! rm -rf .git
! pip install -r requirements.txt

! python setup.py install

clear_output()


### 4.1.3 运行测试

In [6]:

%cd /home/aistudio/PaddleDetection
! python ppdet/modeling/tests/test_architectures.py


/home/aistudio/PaddleDetection
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
[12/28 01:29:14] ppdet.utils.checkpoint INFO: Skipping import of the encryption module.
W1228 01:29:14.668771 483476 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.8
.......
----------------------------------------------------------------------
Ran 7 tests in 7.672s

OK


## 4.2 数据准备

In [7]:

%cd /home/aistudio/
! ls -l


/home/aistudio
total 383
drwxr-xr-x  1 aistudio aistudio   4096 Dec 27 06:17 PaddleDetection
drwxr-xr-x 16 aistudio aistudio   4096 Oct 10 16:18 PaddleDetection-release-2.8.1
drwxr-xr-x  1 aistudio aistudio   4096 Dec 28 01:25 conf
drwxrwxrwx  1 aistudio aistudio   4096 Dec 27 21:41 data
-rw-r--r--  1 aistudio aistudio 301955 Dec 26 08:06 demo.jpg
drwxr-xr-x  4 aistudio aistudio   4096 Dec 26 06:15 external-libraries
-rw-r--r--  1 aistudio aistudio  52850 Dec 28 01:29 main.ipynb
drwxr-xr-x  4 aistudio aistudio   4096 Dec 28 00:47 user_data
drwxr-xr-x  1 aistudio aistudio   4096 Dec 28 01:14 work


In [8]:

! rm -rf user_data && mkdir -p user_data/sample
! cd user_data/sample && unzip -q /home/aistudio/data/datasets/364706/DetectionSample/data.zip

! ls -l user_data/sample
! echo "test" && ls -l user_data/sample/test/*|wc -l
! echo "train" && ls -l user_data/sample/train/*|wc -l
! echo "train_mask" && ls -l user_data/sample/train_mask/*|wc -l


total 92
drwxr-xr-x 2 aistudio aistudio 12288 Dec 28 01:29 test
drwxr-xr-x 2 aistudio aistudio 36864 Dec 28 01:29 train
drwxr-xr-x 2 aistudio aistudio 45056 Dec 28 01:29 train_mask
test
366
train
751
train_mask
751


## 4.3 数据处理
### 4.3.1 路径初始化

In [9]:

"""
user_data/temp/work,  json
user_data/temp/temp0, train mask full
user_data/temp/temp1, train full
user_data/temp/temp2, train mask rectangle
user_data/temp/temp3, test full
"""
! rm -rf user_data/temp/
! mkdir -p user_data/temp/work
! mkdir -p user_data/temp/temp0
! mkdir -p user_data/temp/temp1
! mkdir -p user_data/temp/temp2
! mkdir -p user_data/temp/temp3
! tree user_data/temp


user_data/temp
├── temp0
├── temp1
├── temp2
├── temp3
└── work

5 directories, 0 files


### 4.3.2 开始处理

In [10]:

#
import os
import sys
import json
import random
import cv2 as cv
import numpy as np
import pandas as pd
from tqdm import tqdm

dt = "user_data/sample"
ot = "user_data/temp/"


N = 1550
P = 125


# 
classl = {"cheat": 0, "pmiss": 1}

rjb = {
    "info": {
        "description": "COCO Dataset",
        "url": "/",
        "version": "1.0",
        "year": 2025,
        "contributor": "IvanAXu",
        "date_created": "2025/12/25"
    },
    "licenses": [
        {
            "url": "/",
            "id": 1,
            "name": "MIT License"
        }
    ],
    "categories": [
        {
            "id": classv, 
            "name": f"000{classv}"[-3:], 
            "supercategory": classk
        } for classk, classv in classl.items()
    ]
}
print(rjb)




nnn = 0
annotN = 0
dflip = {
    0: "N", 
    1: -1, 
    2: 0, 
    3: 1, 
    4: "M2", 
    5: "MF", 
    6: "blur", 
    7: "chicken", 
    8: "jely"
}

# record: imagel, imagen, annotl, annotn
record = {}
recordp = "/home/aistudio/user_data/temp/temp1"
for _n in range(3):
    record[f"imagel{_n}"] = []
    record[f"annotl{_n}"] = []


# train
for _i, i in tqdm(
    enumerate(os.listdir(f"{dt}/train_mask")), total=751,
):
    if ".png" in i:
        i = i.replace(".png", "")
        # TODO: 
        _n = random.choice(list("0000000012"))
        
        for _itype in [0, 1, 2, 3, 4, 5, 6, 7, 8]:
            l1, l2 = [], []
            _type = dflip[_itype]
            
            img0 = np.random.randint(0, 1, size=(N,N,3), dtype=np.uint8)
            img0[:, :] = P

            imgb = cv.imread(f"{dt}/train/{i}.jpg")
            imgm = cv.imread(f"{dt}/train_mask/{i}.png")
            i_y, i_x, _ = imgm.shape

            if _type in [-1, 0, 1]:
                imgb = cv.flip(imgb, _type)
                imgm = cv.flip(imgm, _type)
            elif _type == "chicken":
                g = 2
                
                i_yg, i_xg = i_y//g, i_x//g
                c_y = random.randint(i_yg, i_y)
                c_x = random.randint(i_xg, i_x)
                imgb = imgb[:c_y, :c_x]
                imgm = imgm[:c_y, :c_x]
                i_y, i_x, _ = imgm.shape
            elif _type == "blur":
                # imgb = cv.blur(imgb, (2, 2))
                g = 2
                
                i_yg, i_xg = i_y//g, i_x//g
                _imgb = imgb.copy()
                _imgm = imgm.copy()
                imgb[0:i_yg, 0:i_xg] = _imgb[i_yg:i_yg*g, i_xg:i_xg*g]
                imgb[i_yg:i_yg*g, i_xg:i_xg*g] = _imgb[0:i_yg, 0:i_xg]
                imgm[0:i_yg, 0:i_xg] = _imgm[i_yg:i_yg*g, i_xg:i_xg*g]
                imgm[i_yg:i_yg*g, i_xg:i_xg*g] = _imgm[0:i_yg, 0:i_xg]
            elif _type == "jely":
                g = 2
                
                i_yg, i_xg = i_y//g, i_x//g
                _imgb = imgb.copy()
                _imgm = imgm.copy()
                imgb[0:i_yg, 0:i_xg] = _imgb[i_yg:i_yg*g, i_xg:i_xg*g]
                imgb[i_yg:i_yg*g, i_xg:i_xg*g] = _imgb[0:i_yg, 0:i_xg]
                imgm[0:i_yg, 0:i_xg] = _imgm[i_yg:i_yg*g, i_xg:i_xg*g]
                imgm[i_yg:i_yg*g, i_xg:i_xg*g] = _imgm[0:i_yg, 0:i_xg]
            elif _type == "M2":
                g = 2
                
                i_yg, i_xg = i_y//g, i_x//g
                c_y = random.randint(0, i_y-i_yg)
                c_x = random.randint(0, i_x-i_xg)
                
                imgb[c_y:c_y+i_yg, c_x:c_x+i_xg] = P
                imgm[c_y:c_y+i_yg, c_x:c_x+i_xg] = 0
                l1 = [[c_x, c_y, i_xg, i_yg]]
            elif _type == "MF":
                g = 16
                
                i_yg, i_xg = i_y//g, i_x//g
                c_y = random.randint(0, i_y-i_yg)
                c_x = random.randint(0, i_x-i_xg)
                
                imgb[c_y:c_y+i_yg, c_x:c_x+i_xg] = P
                imgm[c_y:c_y+i_yg, c_x:c_x+i_xg] = 0
                l1 = [[c_x, c_y, i_xg, i_yg]]
            
            
            # save full jpg
            img0[:i_y, :i_x] = imgm
            cv.imwrite(f"{ot}/temp0/_{i}_{_type}.jpg", img0)

            img1 = img0.copy()
            img1[:i_y, :i_x] = imgb
            cv.imwrite(f"{ot}/temp1/_{i}_{_type}.jpg", img1)


            # cut the contours
            imgray = cv.cvtColor(imgm, cv.COLOR_BGR2GRAY)  
            ret, thresh = cv.threshold(imgray, 0, 255, cv.THRESH_BINARY) 
            contours, hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

            img2 = img0.copy()
            img2[::] = 255
            cv.drawContours(img2, contours, -1, (255,0,0), 10)


            # get the rectangle list
            l1 += [[0, i_y, N, N-i_y], [i_x, 0, N-i_x, N]]
            l2 += [list(cv.boundingRect(_)) for _ in contours]
            
            # draw the rectangle list
            for _x, _y, _w, _h in l1:
                cv.rectangle(img2, (_x,_y), (_x+_w,_y+_h), (0,0,255), 5) 

            for _x, _y, _w, _h in l2:
                cv.rectangle(img2, (_x,_y), (_x+_w,_y+_h), (0,255,0), 5) 

            cv.imwrite(f"{ot}/temp2/_{i}_{_type}.jpg", img2)


            # json: imagel, imagen, annotl, annotn
            _i_ = _i*10 + _itype
            record[f"imagel{_n}"].append({
                "license": 1,
                "file_name": f"{recordp}/_{i}_{_type}.jpg",
                "coco_url": f"_{i}_{_type}.jpg",
                "width": N,
                "height": N,
                "date_captured": "2020-02-02 01:23:45",
                "flickr_url": "/",
                "id": _i_
            })


            for b_obj in l1:
                record[f"annotl{_n}"].append({
                    "segmentation": [],
                    "area": b_obj[2] * b_obj[3],
                    "iscrowd": 0,
                    "image_id": _i_,
                    "bbox": b_obj,
                    "category_id": classl["pmiss"], 
                    "id": annotN
                })
                annotN += 1

            for b_obj in l2:
                record[f"annotl{_n}"].append({
                    "segmentation": [],
                    "area": b_obj[2] * b_obj[3],
                    "iscrowd": 0,
                    "image_id": _i_,
                    "bbox": b_obj,
                    "category_id": classl["cheat"], 
                    "id": annotN
                })
                annotN += 1
        
        nnn += 1
        if nnn > 2000:
            break


# 0, 1, 2: "trai", "test", "vals"
rj0 = rjb.copy()
rj0["images"] = record[f"imagel0"]
rj0["annotations"] = record[f"annotl0"]

rj1 = rjb.copy()
rj1["images"] = record[f"imagel1"]
rj1["annotations"] = record[f"annotl1"]

rj2 = rjb.copy()
rj2["images"] = record[f"imagel2"]
rj2["annotations"] = record[f"annotl2"]


# 
_T = {"trai": rj0, "test": rj1, "vals": rj2}

for _t in _T.keys():
    with open(f"{ot}/work/class_{_t}.json", "w") as f:
        for classk, classv in classl.items():
            f.write(f"000{classv}"[-3:]+","+classk+"\n")

for _tk, _tv in _T.items():
    target = json.dumps(_tv, ensure_ascii=False)
    with open(f"{ot}/work/annotations_{_tk}.json", 'w') as f:
        f.write(target)


# test
nnn = 0
test_ot = f"{ot}/temp3"
for i in tqdm(
    os.listdir(f"{dt}/test"), total=366,
):
    if ".jpg" in i:
        i = i.replace(".jpg", "")
        img0 = np.random.randint(0, 1, size=(N,N,3), dtype=np.uint8)
        img0[:, :] = P
        
        imgb = cv.imread(f"{dt}/test/{i}.jpg")
        i_y, i_x, _ = imgb.shape
        
        img0[:i_y, :i_x] = imgb
        cv.imwrite(f"{test_ot}/_{i}.jpg", img0)
        
        nnn += 1
        if nnn > 10:
            break



{'info': {'description': 'COCO Dataset', 'url': '/', 'version': '1.0', 'year': 2025, 'contributor': 'IvanAXu', 'date_created': '2025/12/25'}, 'licenses': [{'url': '/', 'id': 1, 'name': 'MIT License'}], 'categories': [{'id': 0, 'name': '000', 'supercategory': 'cheat'}, {'id': 1, 'name': '001', 'supercategory': 'pmiss'}]}


  3%|▎         | 10/366 [00:00<00:24, 14.35it/s]


## 4.4 自定义训练

### 4.4.1 修改配置yml

* **configs/dataset/coco_detection.yml**

```yml
metric: COCO
num_classes: 80

TrainDataset:
  !COCODataSet
    image_dir: train2017 # 训练集的图片所在文件相对于dataset_dir的路径
    anno_path: annotations/instances_train2017.json # 训练集的标注文件相对于dataset_dir的路径
    dataset_dir: dataset/coco # 数据集所在路径，相对于PaddleDetection路径
    data_fields: ['image', 'gt_bbox', 'gt_class', 'is_crowd']

EvalDataset:
  !COCODataSet
    image_dir: val2017 # 验证集的图片所在文件相对于dataset_dir的路径
    anno_path: annotations/instances_val2017.json # 验证集的标注文件相对于dataset_dir的路径
    dataset_dir: dataset/coco # 数据集所在路径，相对于PaddleDetection路径

TestDataset:
  !ImageFolder
    anno_path: annotations/instances_val2017.json # also support txt (like VOC's label_list.txt) # 标注文件所在文件 相对于dataset_dir的路径
    dataset_dir: dataset/coco # if set, anno_path will be 'dataset_dir/anno_path' # 数据集所在路径，相对于PaddleDetection路径
```

修改为：


In [11]:
! cat /home/aistudio/conf/coco_detection.yml

metric: COCO
num_classes: 2

TrainDataset:
  name: COCODataSet
  image_dir: temp1
  anno_path: work/annotations_trai.json
  dataset_dir: ../user_data/temp
  data_fields: ['image', 'gt_bbox', 'gt_class', 'is_crowd']

EvalDataset:
  name: COCODataSet
  image_dir: temp1
  anno_path: work/annotations_vals.json
  dataset_dir: ../user_data/temp
  allow_empty: true

TestDataset:
  name: ImageFolder
  anno_path: work/annotations_test.json
  dataset_dir: ../user_data/temp



* **configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.yml**

```yml
_BASE_: [
  '../datasets/coco_detection.yml',
  '../runtime.yml',
  '_base_/optimizer_1x.yml',
  '_base_/faster_rcnn_r50_fpn.yml',
  '_base_/faster_fpn_reader.yml',
]
weights: output/faster_rcnn_r50_fpn_1x_coco/model_final
```

修改为：



In [12]:
! cat /home/aistudio/conf/faster_rcnn_r50_fpn_1x_coco.yml

_BASE_: [
  '../datasets/coco_detection.yml',
  '../runtime.yml',
  '_base_/optimizer_1x.yml',
  '_base_/faster_rcnn_r50_fpn.yml',
  '_base_/faster_fpn_reader.yml',
]
weights: output/faster_rcnn_r50_fpn_1x_coco/model_final


* **configs/faster_rcnn/_base_/optimizer_1x.yml**

```yml
epoch: 12

LearningRate:
  base_lr: 0.01
  schedulers:
  - !PiecewiseDecay
    gamma: 0.1
    milestones: [8, 11]
  - !LinearWarmup
    start_factor: 0.1
    steps: 1000

OptimizerBuilder:
  optimizer:
    momentum: 0.9
    type: Momentum
  regularizer:
    factor: 0.0001
    type: L2
```

修改为：


In [13]:
! cat /home/aistudio/conf/optimizer_1x.yml

epoch: 3

LearningRate:
  base_lr: 0.001
  schedulers:
  - !PiecewiseDecay
    gamma: 0.1
    milestones: [8, 11]
  - !LinearWarmup
    start_factor: 0.1
    steps: 1000

OptimizerBuilder:
  optimizer:
    momentum: 0.9
    type: Momentum
  regularizer:
    factor: 0.0001
    type: L2


* **configs/runtime.yml**

```yml
use_gpu: true
use_xpu: false
use_mlu: false
use_npu: false
use_gcu: false
log_iter: 20
save_dir: output
snapshot_epoch: 1
print_flops: false
print_params: false

# Exporting the model
export:
  post_process: True  # Whether post-processing is included in the network when export model.
  nms: True           # Whether NMS is included in the network when export model.
  benchmark: False    # It is used to testing model performance, if set `True`, post-process and NMS will not be exported.
  fuse_conv_bn: False
```

修改为：



In [14]:
! cat /home/aistudio/conf/runtime.yml

use_gpu: true
use_xpu: false
use_mlu: false
use_npu: false
use_gcu: false
log_iter: 1000
save_dir: output
snapshot_epoch: 1
print_flops: false
print_params: false

# Exporting the model
export:
  post_process: True  # Whether post-processing is included in the network when export model.
  nms: True           # Whether NMS is included in the network when export model.
  benchmark: False    # It is used to testing model performance, if set `True`, post-process and NMS will not be exported.
  fuse_conv_bn: False


In [15]:

# 自conf复制文件
! cp -f /home/aistudio/conf/coco_detection.yml /home/aistudio/PaddleDetection/configs/datasets/coco_detection.yml
! cp -f /home/aistudio/conf/faster_rcnn_r50_fpn_1x_coco.yml /home/aistudio/PaddleDetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.yml
! cp -f /home/aistudio/conf/optimizer_1x.yml /home/aistudio/PaddleDetection/configs/faster_rcnn/_base_/optimizer_1x.yml
! cp -f /home/aistudio/conf/runtime.yml /home/aistudio/PaddleDetection/configs/runtime.yml


### 4.4.2 开始训练

* 硬件信息

CPU: 2核
AI加速卡: Tesla V100-SXM2-32GB
总显存: 16 GB
总内存: 16 GB
总硬盘: 100 GB

* 环境配置

Python: 3.10.10

* 框架信息

框架版本: PaddlePaddle 3.2.2

训练时，CPU：80%，内存：59%，AI加速卡：38%，显存：70%，仅供参考。


In [16]:
%cd /home/aistudio/PaddleDetection

! export CUDA_VISIBLE_DEVICES=0
! python tools/train.py -c configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.yml


/home/aistudio/PaddleDetection
[12/28 01:45:30] ppdet.utils.checkpoint INFO: Skipping import of the encryption module.
======================= Modified FLAGS detected =======================
FLAGS(name='FLAGS_curand_dir', current_value='/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/../nvidia/curand/lib', default_value='')
FLAGS(name='FLAGS_cupti_dir', current_value='/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/../nvidia/cuda_cupti/lib', default_value='')
FLAGS(name='FLAGS_cusolver_dir', current_value='/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/../nvidia/cusolver/lib', default_value='')
FLAGS(name='FLAGS_nccl_dir', current_value='/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/../nvidia/nccl/lib', default_value='')
FLAGS(name='FLAGS_nvidia_package_dir', current_value='/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/../nvidia', default_value='')

### 4.4.3 模型推理

推理时，CPU：40%，内存：40%，AI加速卡：0%，显存：24%，仅供参考。

In [21]:

%cd /home/aistudio/PaddleDetection

! export CUDA_VISIBLE_DEVICES=0
! python tools/infer.py -c configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.yml -o use_gpu=true weights=output/model_final.pdparams --infer_img=../work/demo-input.jpg

# display(Image(filename="output/demo.jpg"))


/home/aistudio/PaddleDetection
[12/28 02:47:05] ppdet.utils.checkpoint INFO: Skipping import of the encryption module.
W1228 02:47:06.342801 744419 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.8
[12/28 02:47:13] ppdet.utils.checkpoint INFO: Finish loading model weights: output/model_final.pdparams
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.83s/it]
[12/28 02:47:17] ppdet.engine INFO: Detection bbox results save in output/demo-input.jpg



<div>
<img src="https://ai-studio-static-online.cdn.bcebos.com/1c97c83e2ec148f4b1f96172588849af64f52c03e8b04035ab4eb366ece72108" width="400">
<img src="https://ai-studio-static-online.cdn.bcebos.com/17224909350c45eca9bb6eabf87fbba775614416bcdf4799824b732f08a797ea" width="400">
</div>

**从示例结果上看，效果很好！！！**

**在工业化场景下，我们也进行了实际测试，模型检测精度有6%的提升，泛化能力上，有了ERNIE-4.5-VL-28B-A3B的加持，对新图片的检测覆盖也提升10%+**

**后续，我们会进一步跟进它的优化，尽请期待！**
